<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Keras_WithDeeplizard/blob/master/9_Save_And_Load_A_Model_With_TensorFlow's_Keras_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Save And Load A Model With TensorFlow's Keras API
在本集中，我们将演示如何保存和加载tf.keras.Sequential神经网络。

保存Keras模型有几种不同的方法。 多种机制各自以不同的方式保存模型，因此我们将对其进行一一学习。

在前面的章节中，我们创建并训练了这个模型。我们将使用这个模型来演示不同的保存和加载技术。


In [ ]:
# 准备数据
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

train_labels = []
train_samples = []

# 生成数据
for i in range(50):
    # 大约5%的年轻人确实经历过副作用
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(1)

    # 大约5%的老年人没有经历过副作用
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(0)

for i in range(1000):
    # 大约95%的年轻人没有经历过副作用
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(0)

    # 大约95%的老年人确实经历过副作用
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(1)

train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
train_labels,train_samples = shuffle(train_labels,train_samples)
print(train_samples.shape)

# 通过将每个特征缩放到给定的范围来转换特征。
scaler = MinMaxScaler(feature_range=(0,1))

scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))
# 我们只是根据fit_transform（）函数默认情况下不接受一维数据的情况，将数据重塑为2D。
print(train_samples.reshape(-1,1).shape)


(2100,)
(2100, 1)


In [ ]:
# 创建并且编译模型
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

model = Sequential([
    Dense(units=16, input_shape=(1,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=2, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# 训练模型
model.fit(
    x = scaled_train_samples,
    y = train_labels,
    validation_split=0.1,
    batch_size = 10,
    epochs = 30,
    verbose=2
)

Epoch 1/30
189/189 - 0s - loss: 0.2744 - accuracy: 0.9317 - val_loss: 0.3240 - val_accuracy: 0.8762
Epoch 2/30
189/189 - 0s - loss: 0.2728 - accuracy: 0.9370 - val_loss: 0.3235 - val_accuracy: 0.8762
Epoch 3/30
189/189 - 0s - loss: 0.2716 - accuracy: 0.9317 - val_loss: 0.3210 - val_accuracy: 0.8952
Epoch 4/30
189/189 - 0s - loss: 0.2705 - accuracy: 0.9354 - val_loss: 0.3208 - val_accuracy: 0.8952
Epoch 5/30
189/189 - 0s - loss: 0.2693 - accuracy: 0.9376 - val_loss: 0.3211 - val_accuracy: 0.8952
Epoch 6/30
189/189 - 0s - loss: 0.2684 - accuracy: 0.9354 - val_loss: 0.3191 - val_accuracy: 0.8952
Epoch 7/30
189/189 - 0s - loss: 0.2676 - accuracy: 0.9376 - val_loss: 0.3190 - val_accuracy: 0.8952
Epoch 8/30
189/189 - 0s - loss: 0.2667 - accuracy: 0.9386 - val_loss: 0.3189 - val_accuracy: 0.8952
Epoch 9/30
189/189 - 0s - loss: 0.2659 - accuracy: 0.9370 - val_loss: 0.3189 - val_accuracy: 0.8952
Epoch 10/30
189/189 - 0s - loss: 0.2650 - accuracy: 0.9376 - val_loss: 0.3192 - val_accuracy: 0.8952

## 1. 保存和加载整个模型
如果我们想在模型经过训练后保存它的当前状态，以便以后可以使用它，我们可以对模型调用save()函数。对于save()，我们传入文件路径和我们想要保存模型的文件的名称，扩展名为h5。

In [ ]:
# 加载colab云盘数据
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
path = "/content/drive/My Drive/DeepLearning_AI_Course/Deeplizard/Keras - Python Deep Learning Neural Network API/models"

# 更改当前系统的路径
os.chdir(path)
print(os.listdir())

# 获得当前路径
os.getcwd()

[]


'/content/drive/My Drive/DeepLearning_AI_Course/Deeplizard/Keras - Python Deep Learning Neural Network API/models'

In [ ]:
model.save('./9_medical_trial_model.h5')

这种保存方法将保存有关模型的所有信息，包括架构，权重，优化器，优化器的状态，学习率，损失等。

**注意：**如果您愿意，此函数还可以将模型另存为Tensorflow的SavedModel。

现在我们已经保存了这个模型，我们可以在以后加载这个模型。

为此，我们首先导入load_model（）函数。 然后，我们可以通过指向磁盘上保存的模型来调用函数以加载模型。

In [ ]:
from tensorflow.keras.models import load_model
new_model = load_model('./9_medical_trial_model.h5')

我们可以通过调用summary()并在模型上获取weights()来验证加载的模型与保存的模型具有相同的体系结构和权重。

In [ ]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                32        
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


我们还可以通过在加载的模型上调用model.optimizer和model.loss来检查有关模型的属性，例如优化器和损失，并将结果与​​先前保存的模型进行比较。

In [ ]:
print(new_model.optimizer)
print(model.optimizer)
print(new_model.loss)
print(model.loss)
print(new_model.weights[0])
print(model.weights[0])

<function sparse_categorical_crossentropy at 0x7fd527002158>
sparse_categorical_crossentropy
<tf.Variable 'dense/kernel:0' shape=(1, 16) dtype=float32, numpy=
array([[-0.12902698,  0.6980749 ,  0.75187796, -0.47413838, -0.45131892,
        -0.29108244, -0.07799387,  0.3191673 ,  0.268593  , -0.22947463,
         0.38918188, -0.32091162, -0.4051624 , -0.4381077 ,  0.37199572,
        -0.1618033 ]], dtype=float32)>
<tf.Variable 'dense/kernel:0' shape=(1, 16) dtype=float32, numpy=
array([[-0.12902698,  0.6980749 ,  0.75187796, -0.47413838, -0.45131892,
        -0.29108244, -0.07799387,  0.3191673 ,  0.268593  , -0.22947463,
         0.38918188, -0.32091162, -0.4051624 , -0.4381077 ,  0.37199572,
        -0.1618033 ]], dtype=float32)>


这是保存和加载模型的最全面的方法。

## 2. 只保存和加载模型架构
还有另一种方法，我们只保存模型的架构。这不会节省模型权重、配置、优化器、损失或其他任何东西。这只保存了模型的架构。

我们可以通过调用model.to_json（）来实现。 这会将模型的架构另存为JSON字符串。 如果我们打印出字符串，我们可以确切看到它的样子。


In [ ]:
json_string = model.to_json()
json_string

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 1], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_input"}}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "batch_input_shape": [null, 1], "dtype": "float32", "units": 16, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 32, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_r

现在我们已经保存了这个模型结构，我们可以从它创建一个新的模型。首先，我们将从json函数中导入所需的模型，然后我们可以加载模型架构。

In [ ]:
from tensorflow.keras.models import model_from_json
model_architecture = model_from_json(json_string)

通过打印模型摘要，我们可以验证新模型具有与先前保存的模型相同的体系结构。

In [ ]:
model_architecture.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                32        
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 66        
Total params: 642
Trainable params: 642
Non-trainable params: 0
_________________________________________________________________


**注意:**我们也可以使用相同的方法在YAML字符串之间来回保存模型架构。 为此，我们以与调用json函数相同的方式使用函数to_yaml（）和model_from_yaml（）。

## 3. 保存和加载模型的权重
我们将讨论的最后一种保存机制仅保存模型的权重。

为此，我们可以调用model.save_weights（）并传入路径和文件名，以将权重保存为h5扩展名。

In [ ]:
model.save_weights('./9_my_model_weights.h5')

稍后，我们可以将保存的权重加载到新模型中，但是在保存权重之前，新模型需要具有与旧模型相同的体系结构。

In [ ]:
model2 = Sequential([
    Dense(units=16, input_shape=(1,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=2, activation='softmax')
])

In [ ]:
model2.load_weights('./9_my_model_weights.h5')
print(model.weights[0])
print(model2.weights[0])

<tf.Variable 'dense/kernel:0' shape=(1, 16) dtype=float32, numpy=
array([[-0.12902698,  0.6980749 ,  0.75187796, -0.47413838, -0.45131892,
        -0.29108244, -0.07799387,  0.3191673 ,  0.268593  , -0.22947463,
         0.38918188, -0.32091162, -0.4051624 , -0.4381077 ,  0.37199572,
        -0.1618033 ]], dtype=float32)>
<tf.Variable 'dense_3/kernel:0' shape=(1, 16) dtype=float32, numpy=
array([[-0.12902698,  0.6980749 ,  0.75187796, -0.47413838, -0.45131892,
        -0.29108244, -0.07799387,  0.3191673 ,  0.268593  , -0.22947463,
         0.38918188, -0.32091162, -0.4051624 , -0.4381077 ,  0.37199572,
        -0.1618033 ]], dtype=float32)>


现在，我们已经了解了如何仅保存模型的权重并将这些权重部署到新模型，如何仅保存架构，然后将其部署到模型，以及如何保存有关模型的所有内容并将其部署到（稍后再说）。 这些保存和加载机制中的每一个在不同情况下都可能有用。